In [1]:
pip install inseq

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-2.0.0-cp39-cp39-manylinux1_x86_64.whl (619.9 MB)


  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.2 requires torch==2.0.1, but you have torch 2.0.0 which is incompatible.
torchaudio 2.0.2 requires torch==2.0.1, but you have torch 2.0.0 which is incompatible.
You should consider upgrading via the '/sw/arch/Centos8/EB_production/2021/software/Python/3.9.5-GCCcore-10.3.0/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import inseq
import torch
from inseq.attr.step_functions import probability_fn, StepFunctionArgs

# Simplified implementation of inseq.attr.step_functions.contrast_prob_diff_fn
# Works only for encoder-decoder models!
def example_prob_diff_fn(
    # Default arguments for all step functions
    args: StepFunctionArgs,
    # Extra arguments for our use case
    contrast_ids,
    contrast_attention_mask,
):
    """Custom attribution function returning the difference between next step probability for
    candidate generation vs. a contrastive alternative, answering the question "Which features
    were salient in deciding to pick the selected token rather than its contrastive alternative?"

    Extra args:
        contrast_ids: Tensor containing the ids of the contrastive input to be compared to the
            regular one.
        contrast_attention_mask: Tensor containing the attention mask for the contrastive input
    """
    # We truncate contrastive ids and their attention map to the current generation step
    device = args.attribution_model.device
    len_inputs = args.decoder_input_ids.shape[1]
    contrast_decoder_input_ids = contrast_ids[:, : len_inputs].to(device)
    contrast_decoder_attention_mask = contrast_attention_mask[:, : len_inputs].to(device)
    # We select the next contrastive token as target
    contrast_target_ids = contrast_ids[:, len_inputs].to(device)
    # Forward pass with the same model used for the main generation, but using contrastive inputs instead
    contrast_output = args.attribution_model.model(
        inputs_embeds=args.encoder_input_embeds,
        attention_mask=args.encoder_attention_mask,
        decoder_input_ids=contrast_decoder_input_ids,
        decoder_attention_mask=contrast_decoder_attention_mask,
    )
    # Return the prob difference as target for attribution
    model_probs = probability_fn(args)
    args.forward_output = contrast_output
    args.target_ids = contrast_target_ids
    contrast_probs = probability_fn(args)
    return model_probs - contrast_probs
# Register the function defined above
# Since outputs are still probabilities, contiguous tokens can still be aggregated using product
inseq.register_step_function(
    fn=example_prob_diff_fn,
    identifier="example_prob_diff",
    aggregate_map={"span_aggregate": lambda x: x.prod(dim=1, keepdim=True)},
)



In [ ]:
attribution_model = inseq.load_model("gpt2", "occlusion")

# Pre-compute ids and attention map for the contrastive target
contrast = attribution_model.encode("Can you stop the dog from running?", as_targets=True)

# Perform the contrastive attribution:
# Regular (forced) target -> "Non posso crederci."
# Contrastive target      -> "Non posso crederlo."
# contrast_ids & contrast_attention_mask are kwargs defined in the function definition
out = attribution_model.attribute(
    "Can you stop the dog from",
    "Can you stop the dog from barking?",
    attributed_fn="example_prob_diff",
    contrast_ids=contrast.input_ids,
    contrast_attention_mask=contrast.attention_mask,
    attribute_target=True,
    # We also visualize the step score
    step_scores=["example_prob_diff"]
)

# Weight attribution scores by the difference in logits
out.weight_attributions("example_prob_diff")
out.show()

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

ValueError: Decoder-only models should use tokenization as source only.

In [3]:
import inseq

sentence = "Can you stop the dog from"
target = "barking"
foil = " walking"
model = inseq.load_model("gpt2", "input_x_gradient")
normalize = True


In [4]:
model.attribute(
    sentence,
    generation_args={"max_new_tokens": 2},
    attributed_fn="probability",
    n_steps=500,
    step_scores=["probability"],
    internal_batch_size=50
).show()

/home/scur0635/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Unused arguments during attribution: {'n_steps': 500, 'internal_batch_size': 50}
Attributing with input_x_gradient...: 100%|██████████| 8/8 [00:00<00:00, 11.08it/s]


,Ġbarking,?
Can,0.079,0.191
Ġyou,0.102,0.096
Ġstop,0.069,0.07
Ġthe,0.445,0.074
Ġdog,0.169,0.108
Ġfrom,0.136,0.036
Ġbarking,,0.425
?,,
probability,0.142,0.233


In [204]:
attribution_target = model.attribute(
    sentence,
    sentence+target,
    n_steps=500,
    internal_batch_size=50
)

attribution_foil = model.attribute(
    sentence,
    sentence+foil,
    n_steps=500,
    internal_batch_size=50
)

Attributing with input_x_gradient...: 100%|██████████| 7/7 [00:00<00:00, 29.58it/s]


In [205]:
# attributions_target.show()
# attributions_foil.show()

In [206]:
if normalize:
  a = torch.nansum(attributions_target[0].target_attributions)
  b = torch.nansum(attributions_foil[0].target_attributions)
  attributions_target[0].target_attributions /= a
  attributions_foil[0].target_attributions /= b
  # attributions_target.show()
  # attributions_foil.show()

In [207]:
feature_all = torch.FloatTensor.abs_(attributions_target[0].target_attributions)-torch.FloatTensor.abs_(attributions_foil[0].target_attributions)
attribution_contrast = attributions_target
attribution_contrast[0].target_attributions = feature_all

In [208]:
attribution_contrast.show()

,Ġbarking
Can,0.121
Ġyou,0.111
Ġstop,0.101
Ġthe,0.339
Ġdog,0.265
Ġfrom,0.063
Ġbarking,
